In [1]:
import pandas as pd
import geopandas as gpd
import os
import yaml

In [2]:
os.chdir('..')

In [3]:
from pilates.utils import geog

In [4]:
blocks = gpd.read_file('/home/mgardner/austin_blocks.shp')

In [5]:
del blocks['TAZ']

In [6]:
taz = gpd.read_file('/home/mgardner/2015_2045 CAMPO TAZ SHAPE.shp')

In [7]:
with open('settings.yaml') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)

In [8]:
local_crs = settings['local_crs']['austin']

In [9]:
block_to_taz = geog.get_taz_from_block_geoms(blocks, taz, local_crs, 'TAZ')

In [10]:
b2 = blocks.merge(block_to_taz.reset_index(), on=['GEOID'])

In [14]:
b2[['GEOID','zone_id', 'taz_id','TAZ']][b2['GEOID'] == '480219505012027']

,GEOID,zone_id,taz_id,TAZ
2080,480219505012027,1372,4802100001372,1205


In [16]:
b2[['GEOID','TAZ']].to_csv('pilates/polaris/data/austin_blocks_to_taz.csv',index=False)

In [13]:
blocks.shape

(37569, 28)

In [42]:
b2[b2['zone_id'] != b2['TAZ']][['GEOID','zone_id', 'taz_id','TAZ']]

,GEOID,zone_id,taz_id,TAZ
33,480219505023024,1167,4802100001371,1371
245,480219505012028,1372,4802100001372,1205
279,480219501001024,1388,4802100001212,1368
364,480219503005099,2191,4802100001382,1386
371,480219506001199,1141,4802100001183,1183
...,...,...,...,...
37382,484910215072012,109,4849100001412,1412
37402,484910206031006,1847,4849100001946,113
37427,484910210002034,888,4849100000887,887
37487,484910207031008,1355,4849100001320,1320


In [43]:
zones_gdf = taz

In [44]:
blocks_gdf = blocks

In [45]:

zones_gdf = zones_gdf[~zones_gdf['geometry'].is_empty]

In [46]:
zones_gdf = zones_gdf.to_crs(local_crs)
blocks_gdf = blocks_gdf.to_crs(local_crs)

In [47]:
zones_gdf['zone_area'] = zones_gdf.geometry.area

In [48]:
within = gpd.sjoin(blocks_gdf, zones_gdf.reset_index(), how='inner', op='within')

In [ ]:
within = within.sort_values(['GEOID', 'zone_area'])

In [55]:
within = within.sort_values(['GEOID', 'zone_area'])

In [56]:
within = within.drop_duplicates('GEOID', keep='first')

In [72]:
within[within['GEOID'] == '480219505012027']

,GEOID,STATE,COUNTY_left,TRACT_left,BLKGRP,BLOCK,CENTLAT,CENTLON,MTFCC,x,...,D_QRTROP45,D_HALFOP45,AUTODEST45,AUTOORIT45,GQPOP45,TOTEMP15__,POP15__POP,POP45__POP,area_type__right,zone_area
2080,480219505012027,48,021,950501,2,2027,+30.1482475,-097.3597143,G5040,-97.359714,...,0.0,0.0,0,14,0.0,0,0,2074,1,2.650853e+07


In [60]:
block_to_taz_results = pd.concat((
    block_to_taz_results, within[['GEOID', 'TAZ']]))

In [61]:
unassigned_mask = ~blocks_gdf['GEOID'].isin(block_to_taz_results['GEOID'])

In [73]:
intx = gpd.overlay(
    blocks_gdf, zones_gdf.reset_index(),
    how='intersection')

In [74]:
intx['intx_area'] = intx['geometry'].area
intx = intx.sort_values(['GEOID', 'intx_area'], ascending=False)
intx = intx.drop_duplicates('GEOID', keep='first')

In [78]:
intx[intx['GEOID'] == '480219504003004']['TAZ']

3528    1155
Name: TAZ, dtype: int64

In [70]:
blocks_gdf[blocks_gdf['GEOID'] == '480219505012027']

,GEOID,STATE,COUNTY,TRACT,BLKGRP,BLOCK,CENTLAT,CENTLON,MTFCC,x,...,value_impu,place_id,taz_id,zone_id,proportion,TOTEMP,TOTPOP,TOTACRE,area_type_,geometry
2080,480219505012027,48,021,950501,2,2027,+30.1482475,-097.3597143,G5040,-97.359714,...,1,None,4802100001372,1372,0.371152,8.0,0.0,152.10756,0.131486,"POLYGON ((757021.375 4257718.377, 757068.527 4..."


In [71]:
block_to_taz_results[block_to_taz_results['GEOID'] == '480219505012027']

,GEOID,TAZ
2080,480219505012027,1205
2080,480219505012027,1205
